In [33]:
import cv2
import os
import numpy as np
from keras.models import model_from_json

In [34]:
def load_model(filename, verbose=False):
    json_file = open(filename + '.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    loaded_model.load_weights(filename + ".h5")
    if verbose:
        print("The following model " + filename + " has been loaded")
    return loaded_model

In [35]:
def classify_uploded_image(model, image):
    lables = ["Angry", "Disgust", "Fear", "Happy", "Sad", "Surprise", "Neutral"]
    image_copy = np.copy(image)
    gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    faces = faceCascade.detectMultiScale(
        gray_image,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(48, 48)
    )
    face_crop = []
    print("Number of faces: " + str(len(faces)))
    for f in faces:
        x, y, w, h = [ v for v in f ]
        cv2.rectangle(image_copy, (x,y), (x+w, y+h), (255,0,0), 3)
        face_crop.append(gray_image[y:y+h, x:x+w])

    for face in face_crop:
        resized_face = cv2.resize(face,(48,48))
        resized_face = resized_face.astype("float32")/255.
        resized_face = resized_face.reshape(1,48,48,1)
        prediction = model.predict(resized_face)
        
        cv2.imshow('face', face)
        cv2.waitKey(0)
        print(lables[np.argmax(prediction[0])])

In [46]:
image = cv2.imread('img.jpg')
gray_image = cv2.imread('img.jpg',2)
model = load_model("./trained_models/best_40epochs")

In [51]:
classify_uploded_image(model,image,gray_image)

Number of faces: 6


[{'filepath': 'face0.png', 'emotion': 'Happy'},
 {'filepath': 'face1.png', 'emotion': 'Happy'},
 {'filepath': 'face2.png', 'emotion': 'Happy'},
 {'filepath': 'face3.png', 'emotion': 'Happy'},
 {'filepath': 'face4.png', 'emotion': 'Sad'},
 {'filepath': 'face5.png', 'emotion': 'Sad'}]

In [50]:
def classify_uploded_image(model, image_copy, gray_image):
    lables = ["Angry", "Disgust", "Fear", "Happy", "Sad", "Surprise", "Neutral"]
    faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    faces = faceCascade.detectMultiScale(
        gray_image,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(48, 48)
    )
    face_crop = []
    print("Number of faces: " + str(len(faces)))
    for f in faces:
        x, y, w, h = [ v for v in f ]
        cv2.rectangle(image_copy, (x,y), (x+w, y+h), (255,0,0), 3)
        face_crop.append(gray_image[y:y+h, x:x+w])
    index = 0
    items = []
    for face in face_crop:
        resized_face = cv2.resize(face,(48,48))
        resized_face = resized_face.astype("float32")/255.
        resized_face = resized_face.reshape(1,48,48,1)
        prediction = model.predict(resized_face)

        cv2.imshow('face', face)
        cv2.waitKey(0)
        filename = 'face'+str(index)+'.png'
        answer = dict(filepath=filename, emotion=lables[np.argmax(prediction[0])])
        items.append(answer)
        index=index+1
    return items

In [39]:
image.shape

(363, 814, 3)

In [32]:
classify_uploded_image(model, image)

Number of faces: 5
Happy
Happy
Happy
Happy
Sad
